Испытания на классификацию BERT

In [40]:
from collections import Counter
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop = set(stopwords.words("russian"))

def remove_stop_words(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

messages = pd.read_csv("messages.csv", encoding="utf-8")
messages = messages[["id", "text"]]
messages.head()

labels = pd.read_csv("labels.csv", encoding="utf-8")
labels = labels[["id","label"]]
# labels['label'] = labels['label'].map({'other': 0, 'task': 1, 'event':2})
# print(labels.head())

# Убираем stop слова в данных
messages["text"] = messages["text"].map(remove_stop_words)
# labels["label"] = "__label__" + labels["label"].astype(str)
# messages["message_label"] = labels["label"] + " " + messages["text"]
# messages.head()

df = messages
df["label"] = labels["label"].values
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taranyuriy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,text,label
0,2092,насколько понимаю всем рекомендашкам нужны дан...,other
1,8104,странно итмо думает хотя этим многие универы з...,other
2,5821,просто пока ограничимся питером допустим нам н...,other
3,8030,добавил названия таблиц текст сообщения беседе...,other
4,6966,готово,other


In [41]:
df.groupby("label").describe()

id                                                             
        count         mean          std   min      25%     50%      75%   
label                                                                     
event  1006.0  4208.551690  4161.633048  64.0   586.25  1513.0  9325.75  \
other  7432.0  4524.426534  2264.413309   1.0  2725.75  4593.5  6461.25   
task   1004.0  7359.400398  2925.980293   3.0  8369.75  8715.5  8966.25   

               
          max  
label          
event  9577.0  
other  8322.0  
task   9217.0

Перемешаем данные чтобы их было равное количество

In [42]:
df_task = df[df["label"]=="task"]
df_task.shape

df_event = df[df["label"]=="event"]
df_event.shape

df_other = df[df["label"]=="other"]
df_other.shape

df_event_sample = df_event.sample(df_task.shape[0])
df_event_sample.shape

df_other_sample = df_other.sample(df_task.shape[0])
df_other_sample.shape

df_balanced = pd.concat([df_event_sample, df_other_sample,df_task])
df_balanced.shape
df_balanced["label"].value_counts()


label
event    1004
other    1004
task     1004
Name: count, dtype: int64

In [44]:
df_balanced["label_new"] = df_balanced["label"].map({'other': 0, 'task': 1, 'event':2})

Бьём данные на выборки

In [45]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(df_balanced["text"], df_balanced["label_new"], stratify=df_balanced["label_new"])

In [46]:
x_train.head()

3169    первокурсник золотой серебряный бронзовый студ...
8688    пожалуйста изучите найдите хорошее приложение ...
5250    студенты курса магистратуры attention распреде...
8627                        нужно забрал костюм портного.
9013         нужно вымыл окна спальне протер пыль жалюзи.
Name: text, dtype: object

In [6]:
# from transformers import BertTokenizer, TFBertForSequenceClassification,AutoTokenizer, AutoModel
# import tensorflow as tf
# from tensorflow.python.keras.engine import keras_tensor

# # Загрузка предобученной модели BERT
# # model_name = 'cointegrated/rubert-tiny2'

# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# # model_name = 'cointegrated/rubert-tiny2'
# # tokenizer = BertTokenizer.from_pretrained(model_name)
# # model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)
# Загрузка предобученной модели BERT
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Загрузка сохраненной модели (если доступно)
model.load_weights('model_weights.h5')


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'keras.engine'

In [ ]:
# Предобработка и подготовка текста

def preprocess_text(text):
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    return encoded_text

# Классификация текста
def classify_text(text):
    encoded_text = preprocess_text(text)
    input_ids = encoded_text['input_ids']
    attention_mask = encoded_text['attention_mask']

    # Классификация текста
    predictions = model.predict([input_ids, attention_mask])
    predicted_labels = tf.sigmoid(predictions.logits)[0].numpy()

    return predicted_labels

# Пример использования

text = "Это очень интересный фильм!"
predicted_labels = classify_text(text)
print(f"Предсказанные метки: {predicted_labels}")